In [4]:
import json
import pennylane as qml
import pennylane.numpy as np

In [39]:
def U_psi(theta):
    """
    Quantum function that generates |psi>, Zenda's state wants to send to Reece.

    Args:
        theta (float): Parameter that generates the state.

    """
    qml.Hadamard(wires = 0)
    qml.CRX(theta, wires = [0,1])
    qml.CRZ(theta, wires = [0,1])

def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """


    # Put your code here #
    dev = qml.device('default.qubit',wires=2)
    
    @qml.qnode(dev)
    def circuit(theta,alpha,beta):
        
        U_psi(theta)
        qml.RZ(alpha,wires=0)
        qml.RZ(alpha,wires=1)
        qml.RX(beta,wires=0)
        qml.RX(beta,wires=1)
        
        return qml.state()
    
    dev2 = qml.device('default.qubit',wires=2)
    @qml.qnode(dev2)
    def circuit_u(theta):
        U_psi(theta)
        
        return qml.state()
    
    def get_prob(theta, alpha, beta):
        psi = circuit(theta, alpha, beta)
        chi = circuit_u(theta)
        
        prob = np.abs( np.sum(np.conj(chi)*psi) )**2
        return prob
        
    theta_vals = np.linspace(0.,2.*np.pi,101)
    probs = []
    for t_ in theta_vals:
        probs.append(get_prob(t_,alpha,beta))
     
    return np.max(np.array(probs)) >= (1-epsilon)


In [6]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = is_unsafe(*ins)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    
    def bool_to_int(string):
        if string == "True":
            return 1
        return 0

    solution_output = bool_to_int(solution_output)
    expected_output = bool_to_int(expected_output)
    assert solution_output == expected_output, "The solution is not correct."

In [40]:
test_cases = [['[0.1, 0.2, 0.3]', 'True'], ['[1.1, 1.2, 0.3]', 'False'], ['[1.1, 1.2, 0.4]', 'True'], ['[0.5, 1.9, 0.7]', 'True']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1, 0.2, 0.3]'...
Correct!
Running test case 1 with input '[1.1, 1.2, 0.3]'...
Correct!
Running test case 2 with input '[1.1, 1.2, 0.4]'...
Correct!
Running test case 3 with input '[0.5, 1.9, 0.7]'...
Correct!
